# Udiddit, a social news aggregator

Introduction
Udiddit, a social news aggregation, web content rating, and discussion website, is currently using a risky and unreliable Postgres database schema to store the forum posts, discussions, and votes made by their users about different topics.

The schema allows posts to be created by registered users on certain topics, and can include a URL or a text content. It also allows registered users to cast an upvote (like) or downvote (dislike) for any forum post that has been created. In addition to this, the schema also allows registered users to add comments on posts.

Here is the DDL used to create the schema:


```sql
CREATE TABLE bad_posts (
	id SERIAL PRIMARY KEY,
	topic VARCHAR(50),
	username VARCHAR(50),
	title VARCHAR(150),
	url VARCHAR(4000) DEFAULT NULL,
	text_content TEXT DEFAULT NULL,
	upvotes TEXT,
	downvotes TEXT
);

CREATE TABLE bad_comments (
	id SERIAL PRIMARY KEY,
	username VARCHAR(50),
	post_id BIGINT,
	text_content TEXT
);
```

Part I: Investigate the existing schema
As a first step, investigate this schema and some of the sample data in the project’s SQL workspace. Then, in your own words, outline three (3) specific things that could be improved about this schema. Don’t hesitate to outline more if you want to stand out!


1\  table bad_posts : id, topic_id, user_id, title, url, text_content, upvotes, downvotes
2\ table user : user_id, username
3\ table topic : topic_id, topic_name
4\ table bad_comments : id, user_id, post_id, text_content
5\ urdl varchar 100
6\ 


Part II: Create the DDL for your new schema
Having done this initial investigation and assessment, your next goal is to dive deep into the heart of the problem and create a new schema for Udiddit. Your new schema should at least reflect fixes to the shortcomings you pointed to in the previous exercise. To help you create the new schema, a few guidelines are provided to you:


Guideline 
#1: here is a list of features and specifications that Udiddit needs in order to support its website and administrative interface:

a\ Allow new users to register:
Each username has to be unique
Usernames can be composed of at most 25 characters
Usernames can’t be empty
We won’t worry about user passwords for this project


```sql
CREATE TABLE "users" (
  "id" SERIAL PRIMARY KEY,
  "username" VARCHAR(25) NOT NULL,
);

ALTER TABLE "users" ADD CONSTRAINT "user_name_not_null"
    CHECK  (LENGTH(TRIM("username")) > 0);

ALTER TABLE "users" ADD CONSTRAINT "username_unique"
    CHECK  (LENGTH(TRIM("username")) > 0);
```

b\ Allow registered users to create new topics:
Topic names have to be unique.
The topic’s name is at most 30 characters
The topic’s name can’t be empty
Topics can have an optional description of at most 500 characters.


c\ Allow registered users to create new posts on existing topics:
Posts have a required title of at most 100 characters
The title of a post can’t be empty.
Posts should contain either a URL or a text content, but not both.
If a topic gets deleted, all the posts associated with it should be automatically deleted too.
If the user who created the post gets deleted, then the post will remain, but it will become dissociated from that user.


d\ Allow registered users to comment on existing posts:
A comment’s text content can’t be empty.
Contrary to the current linear comments, the new structure should allow comment threads at arbitrary levels.
If a post gets deleted, all comments associated with it should be automatically deleted too.
If the user who created the comment gets deleted, then the comment will remain, but it will become dissociated from that user.
If a comment gets deleted, then all its descendants in the thread structure should be automatically deleted too.


e\ Make sure that a given user can only vote once on a given post:
Hint: you can store the (up/down) value of the vote as the values 1 and -1 respectively.
If the user who cast a vote gets deleted, then all their votes will remain, but will become dissociated from the user.
If a post gets deleted, then all the votes for that post should be automatically deleted too.


Guideline 
#2: here is a list of queries that Udiddit needs in order to support its website and administrative interface. Note that you don’t need to produce the DQL for those queries: they are only provided to guide the design of your new database schema.
List all users who haven’t logged in in the last year.
List all users who haven’t created any post.
Find a user by their username.
List all topics that don’t have any posts.
Find a topic by its name.
List the latest 20 posts for a given topic.
List the latest 20 posts made by a given user.
Find all posts that link to a specific URL, for moderation purposes. 
List all the top-level comments (those that don’t have a parent comment) for a given post.
List all the direct children of a parent comment.
List the latest 20 comments made by a given user.
Compute the score of a post, defined as the difference between the number of upvotes and the number of downvotes


Guideline #3: you’ll need to use normalization, various constraints, as well as indexes in your new database schema. You should use named constraints and indexes to make your schema cleaner.


Guideline #4: your new database schema will be composed of five (5) tables that should have an auto-incrementing id as their primary key.


Part III: Migrate the provided data
Now that your new schema is created, it’s time to migrate the data from the provided schema in the project’s SQL Workspace to your own schema. This will allow you to review some DML and DQL concepts, as you’ll be using INSERT...SELECT queries to do so. Here are a few guidelines to help you in this process:

Topic descriptions can all be empty
Since the bad_comments table doesn’t have the threading feature, you can migrate all comments as top-level comments, i.e. without a parent
You can use the Postgres string function regexp_split_to_table to unwind the comma-separated votes values into separate rows
Don’t forget that some users only vote or comment, and haven’t created any posts. You’ll have to create those users too.
The order of your migrations matter! For example, since posts depend on users and topics, you’ll have to migrate the latter first.
Tip: You can start by running only SELECTs to fine-tune your queries, and use a LIMIT to avoid large data sets. Once you know you have the correct query, you can then run your full INSERT...SELECT query.
NOTE: The data in your SQL Workspace contains thousands of posts and comments. The DML queries may take at least 10-15 seconds to run.

Write the DML to migrate the current data in bad_posts and bad_comments to your new database schema:


In [ ]:
#show table in database
postgres=# \dt

#describe table
postgres=# \d tablename

#show top rows in a table
postgres=# TABLE tablename



# 1  SQL DML: Data Manipulation Language

# 2 Inserting Data: Form One

```sql
INSERT INTO table (column list) VALUES (first row of values), ...
```

```sql
INSERT INTO "movies"("name", "release_date") VALUES
    ('EPISODE 4 - a new hope', '1977-05-25'),
    ('EPISODE 5 - the empire strikes back', '1980-05-17'),
    ('EPISODE 6 - return of the jedi', '1983-05-25');

INSERT INTO "movies"("id", "name", "release_date") VALUES
    (1, 'EPISODE 4 - a new hope', '1977-05-25'),
    (2, 'EPISODE 5 - the empire strikes back', '1980-05-17'),
    (3, 'EPISODE 6 - return of the jedi', '1983-05-25');

INSERT INTO "movies"("name", "release_date") VALUES
    (DEFAULT, 'EPISODE 4 - a new hope', '1977-05-25'),
    (DEFAULT, 'EPISODE 5 - the empire strikes back', '1980-05-17'),
    (DEFAULT, 'EPISODE 6 - return of the jedi', '1983-05-25');

    ```

# 3 Inserting Data: Form Two

```sql
INSERT INTO table_name ("column list in the order it's returned by the SELECT") SELECT … ;

INSERT INTO "categories" ("name") SELECT DISTINCT "category" FROM "books" ;

CREATE TABLE "categories" (
    "id" SERIAL,
    "name" VARCHAR
);

INSERT INTO "categories" ("name") 
    SELECT DISTINCT "category" FROM "posts" ;
```






# 6 Inserting Data: Exercise Solution

```sql
-- Migrate people
INSERT INTO "people" ("first_name", "last_name")
  SELECT "first_name", "last_name" FROM "denormalized_people";

SELECT * FROM "people";

  -- Migrate people's emails using the correct ID
INSERT INTO "people_emails"
SELECT "p"."id", REGEXP_SPLIT_TO_TABLE("dn"."emails", ',')
FROM "denormalized_people" "dn"
JOIN "people" "p" ON (
  "dn"."first_name" = "p"."first_name"
  AND "dn"."last_name" = "p"."last_name"
);
```


# 7 Updating Data in Postgres

```sql
--Create users table

CREATE TABLE users (
    id SERIAL,
    name VARCHAR,
    happiness_level SMALLINT,
    mood VARCHAR
);

-- Insert data into users table

INSERT INTO users (name, happiness_level)
VALUES 
    ('User4', 12),
    ('User1', 50),
    ('User2', 75),
    ('User3', 89);

-- Create posts table

CREATE TABLE posts (
    id SERIAL,
    title TEXT,
    category VARCHAR
);

-- Insert data into posts table

INSERT INTO posts (title, category)
VALUES 
    ('First posts', 'General'),
    ('Second posts', 'Travel'),
    ('Third posts', 'Equipment'),
    ('4th posts', 'Travel'),
    ('lorem ipsum', 'Tips'),
    ('Amother post', 'Travel');

-- Create categories table

CREATE TABLE categories (
    id SERIAL,
    name VARCHAR
);

-- Insert data into categories table

INSERT INTO categories (name)
VALUES 
    ('Travel'),
    ('Equipment'),
    ('General'),
    ('Tips');


-- Update mood based on happiness_level

UPDATE "users" SET "mood" = 'LOW' WHERE "happiness_level" < 33;
UPDATE "users" SET "mood" = 'Average' WHERE "happiness_level" BETWEEN 33 AND 65;
UPDATE "users" SET "mood" = 'Good' WHERE "happiness_level" >= 66;

-- Select all records from the users table

SELECT * FROM users;

-- Update mood to 'Excellent' for all users

UPDATE "users" SET "mood" = 'Excellent';

-- Select all records from the users table

SELECT * FROM users;

-- Update mood to 'Excellent' and set happiness_level to 100 for all users

UPDATE "users" SET "mood" = 'Excellent', "happiness_level" = 100;

-- Select all records from the users table

SELECT * FROM us



-- Add category_id column to posts table

ALTER TABLE "posts" ADD COLUMN "category_id" INTEGER;

-- Display the structure of the posts table

TABLE posts;

-- Update category_id in posts table based on category names

UPDATE "posts" SET "category_id" = (
    SELECT "id" FROM "categories" WHERE "categories"."name" = "posts"."category"
);

-- Display the structure of the posts table

TABLE posts;



```

# 10 Updating Data: Solution

```sql
-- Update the last_name column to be capitalized
UPDATE "people" SET "last_name" =
  SUBSTR("last_name", 1, 1) ||
  LOWER(SUBSTR("last_name", 2));



-- Change the born_ago column to date_of_birth
ALTER TABLE "people" ADD column "date_of_birth" DATE;

UPDATE "people" SET "date_of_birth" = 
  (CURRENT_TIMESTAMP - "born_ago"::INTERVAL)::DATE;

ALTER TABLE "people" DROP COLUMN "born_ago";
```

# 11 Deleting Data in Postgres


The basic syntax for deleting rows from a table is **DELETE FROM table_name WHERE** …. Just like **SELECT** and **UPDATE**, omitting the **WHERE** clause will delete all rows from the table. Again, this is rarely what you want to do! Contrary to **TRUNCATE TABLE**, doing a **DELETE** without a **WHERE** won't allow you to restart the sequence if you have one in your table. More importantly, in a future lesson we'll learn about indexing as a way to make queries perform faster in the presence of large amounts of data. Running TRUNCATE will also clear these indexes, which will further accelerate queries once new data gets inserted in that table.

# 12 Data Manipulation: Transactions

```sql
SELECT
```

# 14 Data Manipulation Exercise: Solution

```sql
-- Do everything in a transaction
BEGIN;


-- Remove all users from New York and California
DELETE FROM "user_data" WHERE "state" IN ('NY', 'CA');


-- Split the name column in first_name and last_name
ALTER TABLE "user_data"
  ADD COLUMN "first_name" VARCHAR,
  ADD COLUMN "last_name" VARCHAR;

UPDATE "user_data" SET
  "first_name" = SPLIT_PART("name", ' ', 1),
  "last_name" = SPLIT_PART("name", ' ', 2);

ALTER TABLE "user_data" DROP COLUMN "name";


-- Change from state to state_id
CREATE TABLE "states" (
  "id" SMALLSERIAL,
  "state" CHAR(2)
);

INSERT INTO "states" ("state")
  SELECT DISTINCT "state" FROM "user_data";

ALTER TABLE "user_data" ADD COLUMN "state_id" SMALLINT;

UPDATE "user_data" SET "state_id" = (
  SELECT "s"."id"
  FROM "states" "s"
  WHERE "s"."state" = "user_data"."state"
);

ALTER TABLE "user_data" DROP COLUMN "state";
```

# 16 Glossary

| Key Term            | Definition |
|---------------------|------------|
| **INSERT ... VALUES** | One form of adding data into a table; used when introducing new data in a table. This data would come from an external source like an application. |
| **INSERT ... SELECT** | One form of adding data into a table; used when taking already existing data from a table and migrating it — most often with some modifications or clean-ups — into an already existing table. |
| **UPDATE**           | Used to update rows of data within a given column with new values. |
| **DELETE**           | Used to delete some portion of data from a table. |
| **BEGIN**            | Starts a transaction. |
| **COMMIT**           | Tells the system to attempt to complete the transaction (make the requested changes). Similar functionality is also achieved with END. |
| **ROLLBACK**         | Tells the system to not commit any changes as part of the current transaction, discarding the changes. |
| **ACID**             | An acronym that describes the transactional guarantees provided by a relational database. |
| **Atomicity**        | The database guarantees that a transaction will either register all the commands in a transaction, or none of them. |
| **Consistency**      | The database guarantees that a successful transaction will leave the data in a consistent state, one that obeys all the rules that you've set up. |
| **Isolation**        | The database guarantees that concurrent transactions don't "see each other" until they are committed. Committing a transaction is a command that tells the database to execute all the commands we passed to it since we started that transaction. |
| **Durability**       | The database guarantees that once it accepts a transaction and returns a success, the changes introduced by the transaction will be permanently stored on disk, even if the database crashes right after the success response. |


```sql

```